In [1]:
import pandas as pd
import unicodedata
import re

In [2]:
df = pd.read_csv("hf://datasets/vishnupriyavr/wiki-movie-plots-with-summaries/wiki_movie_plots_deduped_with_summaries.csv")

In [3]:
df

,Release Year,Title,Origin/Ethnicity,Director,Cast,Genre,Wiki Page,Plot,PlotSummary
0,1901,Kansas Saloon Smashers,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/Kansas_Saloon_Sm...,"A bartender is working at a saloon, serving dr...",Carrie Nation and her followers burst into a s...
1,1901,Love by the Light of the Moon,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/Love_by_the_Ligh...,"The moon, painted with a smiling face hangs ov...","The moon, painted with a smiling face hangs ov..."
2,1901,The Martyred Presidents,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/The_Martyred_Pre...,"The film, just over a minute long, is composed...","The film, just over a minute long, is composed..."
3,1901,"Terrible Teddy, the Grizzly King",American,Unknown,NaN,unknown,"https://en.wikipedia.org/wiki/Terrible_Teddy,_...",Lasting just 61 seconds and consisting of two ...,The first shot is set in a wood during winter ...
4,1902,Jack and the Beanstalk,American,"George S. Fleming, Edwin S. Porter",NaN,unknown,https://en.wikipedia.org/wiki/Jack_and_the_Bea...,The earliest known adaptation of the classic f...,The earliest known adaptation of the classic f...
...,...,...,...,...,...,...,...,...,...
34881,2014,The Water Diviner,Turkish,Director: Russell Crowe,Director: Russell Crowe\r\nCast: Russell Crowe...,unknown,https://en.wikipedia.org/wiki/The_Water_Diviner,"The film begins in 1919, just after World War ...",Russell Crowe's film centres around Australian...
34882,2017,Çalgı Çengi İkimiz,Turkish,Selçuk Aydemir,"Ahmet Kural, Murat Cemcir",comedy,https://en.wikipedia.org/wiki/%C3%87alg%C4%B1_...,"Two musicians, Salih and Gürkan, described the...","Two musicians, Salih and Gürkan, described the..."
34883,2017,Olanlar Oldu,Turkish,Hakan Algül,"Ata Demirer, Tuvana Türkay, Ülkü Duru",comedy,https://en.wikipedia.org/wiki/Olanlar_Oldu,"Zafer, a sailor living with his mother Döndü i...","Zafer, a sailor living with his mother Döndü i..."
34884,2017,Non-Transferable,Turkish,Brendan Bradley,"YouTubers Shanna Malcolm, Shira Lazar, Sara Fl...",romantic comedy,https://en.wikipedia.org/wiki/Non-Transferable...,The film centres around a young woman named Am...,The film centres around a young woman who book...


In [4]:
df.isna().sum()

Release Year           0
Title                  0
Origin/Ethnicity       0
Director               0
Cast                1422
Genre                  0
Wiki Page              0
Plot                   0
PlotSummary            0
dtype: int64

In [7]:
df['Plot'].str.len().describe()

count    34886.000000
mean      2165.034541
std       1817.325247
min         15.000000
25%        716.000000
50%       1656.000000
75%       3376.000000
max      36773.000000
Name: Plot, dtype: float64

In [5]:
df['Plot'].str.split().str.len().describe()

count    34886.000000
mean       372.493206
std        315.753223
min          2.000000
25%        122.000000
50%        284.000000
75%        581.000000
max       6752.000000
Name: Plot, dtype: float64

In [8]:
df['PlotSummary'].str.len().describe()

count    34886.000000
mean       380.977727
std         97.721757
min        160.000000
25%        302.000000
50%        365.000000
75%        452.000000
max        697.000000
Name: PlotSummary, dtype: float64

In [6]:
df['PlotSummary'].str.split().str.len().describe()

count    34886.000000
mean        65.240154
std         16.756229
min         22.000000
25%         52.000000
50%         62.000000
75%         77.000000
max        116.000000
Name: PlotSummary, dtype: float64

In [4]:
new_df = df[df['Plot'].str.split().str.len() > 60]
# new_df = new_df[new_df['Plot'].str.split().str.len() < 500]
new_df = new_df.dropna()  # subset=['Cast']
new_df = new_df[new_df.Cast.str.lower() != 'unknown']
new_df = new_df[new_df.Genre.str.lower() != 'unknown']
new_df = new_df.reset_index(drop=True)
new_df

,Release Year,Title,Origin/Ethnicity,Director,Cast,Genre,Wiki Page,Plot,PlotSummary
0,1907,Daniel Boone,American,Wallace McCutcheon and Ediwin S. Porter,"William Craven, Florence Lawrence",biographical,https://en.wikipedia.org/wiki/Daniel_Boone_(19...,Boone's daughter befriends an Indian maiden as...,"Boone's cabin is attacked by the Indians, who ..."
1,1908,The Adventures of Dollie,American,D. W. Griffith,"Arthur V. Johnson, Linda Arvidson",drama,https://en.wikipedia.org/wiki/The_Adventures_o...,On a beautiful summer day a father and mother ...,A father and mother take their daughter Dollie...
2,1908,The Black Viper,American,D. W. Griffith,D. W. Griffith,drama,https://en.wikipedia.org/wiki/The_Black_Viper,A thug accosts a girl as she leaves her workpl...,A thug accosts a girl as she leaves her workpl...
3,1908,The Call of the Wild,American,D. W. Griffith,Charles Inslee,adventure,https://en.wikipedia.org/wiki/The_Call_of_the_...,A white girl (Florence Lawrence) rejects a pro...,Florence Lawrence was already the most popular...
4,1908,A Christmas Carol,American,Unknown,Tom Ricketts,drama,https://en.wikipedia.org/wiki/A_Christmas_Caro...,No prints of the first American film adaptatio...,No prints of the first American film adaptatio...
...,...,...,...,...,...,...,...,...,...
25194,2011,White as Snow,Turkish,Selim Güneş,"Hakan Korkmaz, Sinem İslamoğlu & Gürkan Piri O...",drama,https://en.wikipedia.org/wiki/White_as_Snow_(f...,Hasan is a twelve-year-old boy living with his...,Hasan is a twelve-year-old boy living in a mou...
25195,2011,Once Upon a Time in Anatolia,Turkish,Nuri Bilge Ceylan,"Yılmaz Erdoğan, Taner Birsel & Ufuk Karaali",drama,https://en.wikipedia.org/wiki/Once_Upon_a_Time...,"Through the night, three cars carry a small gr...",A small group of men search for a buried body ...
25196,2013,Selam,Turkish,Levent Demirkale,"Bucin Abdullah, Selma Alispahic, Tina Cvitanov...",drama,https://en.wikipedia.org/wiki/Selam_(film),The film opens with a Senegalese boy named Kha...,The film opens with a Senegalese boy named Kha...
25197,2017,Olanlar Oldu,Turkish,Hakan Algül,"Ata Demirer, Tuvana Türkay, Ülkü Duru",comedy,https://en.wikipedia.org/wiki/Olanlar_Oldu,"Zafer, a sailor living with his mother Döndü i...","Zafer, a sailor living with his mother Döndü i..."


In [5]:
def clean_text(text):

    text = re.sub(r'<[^>]+>', ' ', text)
    text = re.sub(r'\[.*?\]', ' ', text)
    text = unicodedata.normalize('NFKC', text)

    # Нормализация пробелов
    text = re.sub(r'\s+', ' ', text).strip()

    return text

In [6]:
new_df['PlotSummary'] = new_df['PlotSummary'].apply(clean_text)
new_df

,Release Year,Title,Origin/Ethnicity,Director,Cast,Genre,Wiki Page,Plot,PlotSummary
0,1907,Daniel Boone,American,Wallace McCutcheon and Ediwin S. Porter,"William Craven, Florence Lawrence",biographical,https://en.wikipedia.org/wiki/Daniel_Boone_(19...,Boone's daughter befriends an Indian maiden as...,"Boone's cabin is attacked by the Indians, who ..."
1,1908,The Adventures of Dollie,American,D. W. Griffith,"Arthur V. Johnson, Linda Arvidson",drama,https://en.wikipedia.org/wiki/The_Adventures_o...,On a beautiful summer day a father and mother ...,A father and mother take their daughter Dollie...
2,1908,The Black Viper,American,D. W. Griffith,D. W. Griffith,drama,https://en.wikipedia.org/wiki/The_Black_Viper,A thug accosts a girl as she leaves her workpl...,A thug accosts a girl as she leaves her workpl...
3,1908,The Call of the Wild,American,D. W. Griffith,Charles Inslee,adventure,https://en.wikipedia.org/wiki/The_Call_of_the_...,A white girl (Florence Lawrence) rejects a pro...,Florence Lawrence was already the most popular...
4,1908,A Christmas Carol,American,Unknown,Tom Ricketts,drama,https://en.wikipedia.org/wiki/A_Christmas_Caro...,No prints of the first American film adaptatio...,No prints of the first American film adaptatio...
...,...,...,...,...,...,...,...,...,...
25194,2011,White as Snow,Turkish,Selim Güneş,"Hakan Korkmaz, Sinem İslamoğlu & Gürkan Piri O...",drama,https://en.wikipedia.org/wiki/White_as_Snow_(f...,Hasan is a twelve-year-old boy living with his...,Hasan is a twelve-year-old boy living in a mou...
25195,2011,Once Upon a Time in Anatolia,Turkish,Nuri Bilge Ceylan,"Yılmaz Erdoğan, Taner Birsel & Ufuk Karaali",drama,https://en.wikipedia.org/wiki/Once_Upon_a_Time...,"Through the night, three cars carry a small gr...",A small group of men search for a buried body ...
25196,2013,Selam,Turkish,Levent Demirkale,"Bucin Abdullah, Selma Alispahic, Tina Cvitanov...",drama,https://en.wikipedia.org/wiki/Selam_(film),The film opens with a Senegalese boy named Kha...,The film opens with a Senegalese boy named Kha...
25197,2017,Olanlar Oldu,Turkish,Hakan Algül,"Ata Demirer, Tuvana Türkay, Ülkü Duru",comedy,https://en.wikipedia.org/wiki/Olanlar_Oldu,"Zafer, a sailor living with his mother Döndü i...","Zafer, a sailor living with his mother Döndü i..."


In [11]:
def create_movie_text(row):
    components = []

    components.append(f"Title: {row['Title']}.")
    components.append(f"Release Year: {row['Release Year']}.")
    components.append(f"Director: {row['Director']}.")
    components.append(f"Cast: {row['Cast']}.")
    components.append(f"Genre: {row['Genre']}.")
    components.append(f"Plot: {row['PlotSummary']}") # Plot

    return "\n".join(components)

In [12]:
new_df['movie_text'] = new_df.apply(create_movie_text, axis=1)

In [13]:
print(new_df['movie_text'].iloc[100])

Title: A Modern Musketeer.
Release Year: 1917.
Director: Allan Dwan.
Cast: Douglas Fairbanks, Marjorie Daw.
Genre: adventure.
Plot: D'Artagnan (Douglas Fairbanks) is a modern day Ned Thacker, played by Fairbanks. Ned is born and raised in Kansas by a mother who passes along to him her love of The Three Musketeers. Ned gets into trouble with his (sometimes unwanted) chivalrous attempts to help women.


In [14]:
new_df['movie_text'].str.len().describe()

count    25199.000000
mean       550.142347
std        104.661619
min        292.000000
25%        468.000000
50%        536.000000
75%        626.000000
max       1145.000000
Name: movie_text, dtype: float64

In [16]:
new_df.to_csv('./../data/movies_data.csv', index=False)